# 猫狗识别


In [3]:
import os
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def make_data_set(src_path, dst_path):
    """

    :param src_path:
    :param dst_path:
    :return:
    """
    dataset_trian_cats = os.path.join(dst_path, 'train', 'cats')
    dataset_trian_dogs = os.path.join(dst_path, 'train', 'dogs')
    dataset_valid_cats = os.path.join(dst_path, 'valid', 'cats')
    dataset_valid_dogs = os.path.join(dst_path, 'valid', 'dogs')

    if not os.path.isdir(dataset_trian_cats):
        os.makedirs(dataset_trian_cats)

    if not os.path.isdir(dataset_trian_dogs):
        os.makedirs(dataset_trian_dogs)

    if not os.path.isdir(dataset_valid_cats):
        os.makedirs(dataset_valid_cats)

    if not os.path.isdir(dataset_valid_dogs):
        os.makedirs(dataset_valid_dogs)

    dogs_and_cats = os.listdir(src_path)



    cats = [s for s in dogs_and_cats if s.startswith('cat')]
    dogs = [s for s in dogs_and_cats if s.startswith('dog')]

    train_cats = cats[:int(len(cats) * 0.8)]
    valid_cats = cats[int(len(cats) * 0.8):]
    train_dogs = dogs[:int(len(dogs) * 0.8)]
    valid_dogs = dogs[int(len(dogs) * 0.8):]

    for cat in train_cats:
        os.symlink(os.path.join(src_path, cat), os.path.join(dataset_trian_cats, cat))

    for dog in train_dogs:
        os.symlink(os.path.join(src_path, dog), os.path.join(dataset_trian_dogs, dog))

    for cat in valid_cats:
        os.symlink(os.path.join(src_path, cat), os.path.join(dataset_valid_cats, cat))

    for dog in valid_dogs:
        os.symlink(os.path.join(src_path, dog), os.path.join(dataset_valid_dogs, dog))

src_path = "./cats_and_dogs/dataset"       
dst_path = "./cats_and_dogs" 
make_data_set(src_path, dst_path)

In [8]:
base_dir = './cats_and_dogs'
train_dir = os.path.join(base_dir,'train')
test_dir = os.path.join(base_dir,'valid')

train_cats_dir = os.path.join(train_dir,'cats')
train_dogs_dir = os.path.join(train_dir,'dogs')

validation_cats_dir = os.path.join(test_dir,'cats')
validation_dogs_dir = os.path.join(test_dir,'dogs')

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(64,64,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [6]:
model.compile(loss='binary_crossentropy',
             optimizer=Adam(lr=1e-4),
             metrics=['acc'])

In [9]:
# 数据增强，训练数据更多：减少过拟合
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(64,64),
                                                    batch_size=20,
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(64,64),
                                                    batch_size=20,
                                                    class_mode='binary')

history = model.fit_generator(train_generator,
                             steps_per_epoch=100,
                             epochs=100,
                             validation_data = validation_generator,
                             validation_steps = 50,
                             verbose=2)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


OSError: [Errno 22] Invalid argument: './cats_and_dogs\\train\\dogs\\dog.2123.jpg'

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs,acc,'b',label='Training accuracy')
plt.plot(epochs,val_acc,'r',label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs,loss,'b',label='Training loss')
plt.plot(epochs,val_loss,'r',label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
tf.keras.layers.Dropout(0.5) # 加在全连接层